# Tensorflow: K-nearest Neighbors (applied to Iris data set)
*Rachel Buttry*

Updated: *13 April 2018*

We can re-create the K-nearest neighbors model (for classification) in tensorflow.

The following code was taken from or based off of these examples: 

* [PHYS T480 Scikit-Learn-Intro notebook](https://github.com/gtrichards/PHYS_T480/blob/master/Scikit-Learn-Intro.ipynb)
* [Tensorflow example](https://github.com/tensorflow/models/blob/master/samples/core/get_started/iris_data.py)
* [Knn example 1](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/2_BasicModels/nearest_neighbor.py) 
* [Knn example 2](http://marubon-ds.blogspot.com/2017/09/knn-k-nearest-neighbors-by-tensorflow.html)

**Note:** For this example, I am assuming you are familiar with sklearn and atleast have a general idea how the K-nearest neighbors model works.

Start off by importing our data and splitting it into a training/test sets.

In [1]:
# Importing data thru scikitlearn 
from sklearn.datasets import load_iris
import numpy as np
iris = load_iris()

x_vals = np.array([x[0:4] for x in iris.data])
y_vals = np.array(iris.target)

# train-test split
np.random.seed(1234)
train_perc = 0.5#percent of data used for training
train_indices = np.random.choice(len(x_vals), \
            int(round(len(x_vals) * train_perc)), replace=False)
test_indices =np.array(list(set(range(len(x_vals))) - set(train_indices)))

train_data = x_vals[train_indices]
test_data = x_vals[test_indices]
train_labels = y_vals[train_indices]
test_labels = y_vals[test_indices]

###### K-Nearest Neigbor(s) 
For classification, we take the k nearest data points, look at their classifications, and see which one occurs most frequently in the sample. Image from [Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)

<img src='./images/KnnClassification.svg' width='25%'>

Be aware that the code below uses euclidian distance and uniform weights (which are the ```sklearn.neighbors.KNeighborsClassifier``` defaults). The model can be adjusted to use different distance metrics or have weights based on distance.

In [2]:
k = 5 #number of neigbors we're considering

#import tensorflow and create our session
import tensorflow as tf
sess = tf.Session()

For KNN (with default parameters), here's what we need for each test data point:
1. Calculate the euclidian (L2) distances to the training data points
2. Determine the k nearest data points (in the training data) - these are the neighbors
3. Count up the how many of the neighbors have which label
4. Take the most common label as our prediction for the test data point

**Note:** For step 3, it's a common convention to used a weighted count with the weights  $\frac{1}{d}$, where $d$ is the distance found in step 1. However, we're just using uniform weights for this example. 

In [3]:
# tf Graph Input
xtr = tf.placeholder(tf.float32, name="training_data") 
xte = tf.placeholder(tf.float32, name="testing_data")
ytr = tf.placeholder(tf.float32, name="training_labels")

# Nearest Neighbor calculation using L2 Distance
sqr_dist = tf.reduce_sum(tf.square(tf.subtract(xte, xtr)), reduction_indices=1)
distance = tf.sqrt(sqr_dist)

# nearest k points
_, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
top_k_labels = tf.gather(ytr, top_k_indices)

# get uniform weights
weights = tf.constant(1./k)

ordered_labels, indx, count = tf.unique_with_counts(top_k_labels)


weighted_count = tf.multiply(tf.cast(count, tf.float32), weights)
pred = ordered_labels[tf.argmax(weighted_count)]

#### Time to test it on our data!
Note that there isn't a ```fit()``` stage in this one like in sklearn (see the sklearn used later). This is because each time we pass thru testing data, we're also passing in the training data via the placeholders ```xtr``` and  ```ytr```. You can think of it as the model fitting and predicting at the same time.

In [4]:
accuracy = 0.0

# Initialize the variables (i.e. assign their default value)
#init = tf.global_variables_initializer()

# Run the initializer
#sess.run(init) 

# loop over test data
for i in range(len(test_data)):
    # Prect
    nn_index = sess.run(pred, {xtr:train_data, xte: test_data[i, :], ytr: train_labels})
    
    #we need to extract the answer, because right now, it's just an index
    prediction = int(nn_index)#int for indexing the target names
    actual_label = test_labels[i]
    
    print("Test", i, "Prediction:", iris['target_names'][prediction], \
        "True Class:", iris['target_names'][actual_label])
    # Calculate accuracy
    if prediction == actual_label:
        accuracy += 1./len(test_data)

('Test', 0, 'Prediction:', 'virginica', 'True Class:', 'virginica')
('Test', 1, 'Prediction:', 'virginica', 'True Class:', 'virginica')
('Test', 2, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 3, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 4, 'Prediction:', 'virginica', 'True Class:', 'virginica')
('Test', 5, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 6, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 7, 'Prediction:', 'virginica', 'True Class:', 'virginica')
('Test', 8, 'Prediction:', 'virginica', 'True Class:', 'virginica')
('Test', 9, 'Prediction:', 'virginica', 'True Class:', 'virginica')
('Test', 10, 'Prediction:', 'virginica', 'True Class:', 'virginica')
('Test', 11, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 12, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 13, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 14, 'Prediction:', 'setosa', 'True Class:', 'setosa')
('Test', 15, 'Predictio

In [5]:
print "Accuracy: ", accuracy

Accuracy:  0.96


In [6]:
#visualize out method using tensorgraph
tf.reset_default_graph()#reset graph
writer = tf.summary.FileWriter("./graphs/iris_knn", sess.graph)
writer.close()

The graph looks like this:
<img src='./graphs/iris_knn.png' width='50%'>

It may look complicted, but remember, each node is just a mathematical operation or some action used on an array (such as indexing).

### The scikit-learn way
We can also just use the built-in scikit-learn KNN model. I'll just run it to check if the accuracy is (atleast) comparable.

In [7]:
from sklearn import neighbors

# create the model
knn = neighbors.KNeighborsClassifier(n_neighbors=k)

# fit the model
knn.fit(train_data, train_labels)

# test it on the same test dataset
accuracy_sk = 0.0
for j,a in zip(test_data, test_labels):
    result = knn.predict(j.reshape(1,-1))
    
    print("Prediction:", iris['target_names'][result][0], \
    "True Class:", iris['target_names'][a])
    # Calculate accuracy
    if result == a:
        accuracy_sk += 1./len(test_data)

('Prediction:', 'virginica', 'True Class:', 'virginica')
('Prediction:', 'virginica', 'True Class:', 'virginica')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'virginica', 'True Class:', 'virginica')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'virginica', 'True Class:', 'virginica')
('Prediction:', 'virginica', 'True Class:', 'virginica')
('Prediction:', 'virginica', 'True Class:', 'virginica')
('Prediction:', 'virginica', 'True Class:', 'virginica')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'setosa', 'True Class:', 'setosa')
('Prediction:', 'virginica', 'True Class

In [8]:
print "Accuracy: ", accuracy_sk 

Accuracy:  0.96


The accuracies are the same. 

### Final Notes
The tensorflow model is much lower level, so, in order to use KNN, we needed to have a deep enough understanding of the model to build it from scratch. Knowing this and that KNN is a relatively simple model, we can imagine the detail we'd have to go into to recreate the more complicated models. 

This is a pain that we can't really get around yet. Tensorflow may, at some point, add KNN (and other models) such that they can simply be called, but that is not the case for the current version. For now, if you want to just use off-the-shelf machine learning methods, you're better off using the ones that already exist in sklearn rather than trying to reinvent the wheel.

So then why use tensorflow? Sure, tensorflow may not be good for the common models we're familiar with, but that's not really what it's meant for. Tensorflow really shines when we want to build neural networks and do deep learning. 
